In [1]:
from time import time
import math
import gmpy2
import random

In [2]:
DISTINCT = 37
CHECK = ord('-')-ord('a') # for '-'
FIX = 26 # for '-'
FIX_NUM = 27 # for number '0' and so on...
KEY_LEN = 512

In [3]:
SMALL_PRIMES_UPTO = 100000
SMALL_PRIMES = [i for i in range(SMALL_PRIMES_UPTO) if gmpy2.is_prime(i)]
def remove_small_primes(n):
    for prime in SMALL_PRIMES:
        while n%prime==0:
            n  //= prime
    return n

In [4]:
def is_strong_prime(p):
    if not gmpy2.is_prime(p): return False
    
    # p-1
    q = remove_small_primes(p-1)
    if not gmpy2.is_prime(q): return False
    
    # q-1
    if not gmpy2.is_prime(remove_small_primes(q-1)): return False

    #p+1
    if not gmpy2.is_prime(remove_small_primes(p+1)): return False
    
    return True

In [5]:
def generate_strong_prime(length=512):
    
    low = 2**length
    high = 2**(length+1) - 1
    
    strong_prime_candidate = int(gmpy2.next_prime(random.randint(low, high)))
    while not is_strong_prime(strong_prime_candidate):
#         strong_prime_candidate = int(gmpy2.next_prime( strong_prime_candidate + 1 ))
        strong_prime_candidate = int(gmpy2.next_prime(random.randint(low, high)))
    
    return strong_prime_candidate

In [6]:
def generate_strong_prime_2(length=512):
    
    low = 2**length
    high = 2**(length+1) - 1
    
    # Pn = 2*q + 1
    # also: Pn > (Pn-1 + Pn+1)/2 
    Pn0 = int(gmpy2.next_prime(random.randint(low, high)))
    Pn1 = int(gmpy2.next_prime( Pn0 + 1 ))
    Pn2 = int(gmpy2.next_prime( Pn1 + 1 ))
    
    q = (Pn1-1)//2
    
    while not( Pn1 > (Pn0+Pn2)//2 and gmpy2.is_prime(q) ):
        
        Pn0 = int(gmpy2.next_prime(random.randint(low, high)))
        Pn1 = int(gmpy2.next_prime( Pn0 + 1 ))
        Pn2 = int(gmpy2.next_prime( Pn1 + 1 ))

        q = (Pn1-1)//2
    return Pn1

In [7]:
def find_block_size(num_diff_chars, n):
    block_size = int(math.log(n, num_diff_chars))

    if num_diff_chars**block_size >= n:
        return block_size-1
    else:
        return block_size

In [8]:
def encrypt_RSA(pl_text, key, n):
    ci_text = ''
    r = find_block_size(DISTINCT, n)
    s = r+1
    for i in range(0, len(pl_text), r):
        block = pl_text[i:i+r]
        M = 0
        for j, ch in enumerate(block):
            if ch=='-':
                M += (FIX)*(DISTINCT**j)
            elif '0' <= ch <= '9':
                M += (ord(ch) - ord('0') + FIX_NUM)*(DISTINCT**j)
            else:
                M += (ord(ch) - ord('a'))*(DISTINCT**j)
        
        C = pow(M, key, n)
        
        for j in range(s-1,-1,-1):
            temp = C // (DISTINCT**j)
            C -= temp*(DISTINCT**j)
            
            # for Numbers
            if FIX_NUM <= temp <= FIX_NUM+9:
                ci_text += chr(temp-FIX_NUM + ord('0'))
            else:
                if temp == FIX: temp = CHECK # for '-'
                ci_text += chr(temp+ord('a'))

    return ci_text

In [9]:
CA_p = generate_strong_prime_2(KEY_LEN)
CA_q = generate_strong_prime_2(KEY_LEN)
CA_n = CA_p * CA_q
CA_phi_n = (CA_p-1) * (CA_q-1)

low = 2**(KEY_LEN-1)
high = CA_phi_n - 1

CA_e = int(gmpy2.next_prime(random.randint(low, high)))
g, s, t = gmpy2.gcdext(CA_e, CA_phi_n)
while g != 1:
    CA_e = int(gmpy2.next_prime(random.randint(low, high)))
    g, s, t = gmpy2.gcdext(CA_e, CA_phi_n)
CA_d = s%CA_phi_n

open('public_CA.txt', 'w+').write(str(CA_e) + ' ' + str(CA_n))

619

In [10]:
def generate_keys_CA(N=2):
    """
        N = No. of people in group for whom public and private keys have to be generated.
        
        Outputs 1 file with everyone's public key.
        and N files with their respective private key.
    """
    # Remove all old keys
    open('public_dir.txt', 'w+').close()

    for i in range(1,N+1):
        p = generate_strong_prime_2(KEY_LEN)
        q = generate_strong_prime_2(KEY_LEN)
        n = p*q
        phi_n = (p-1)*(q-1)
        
        low = 2**(KEY_LEN-1)
        high = phi_n - 1

        e = int(gmpy2.next_prime(random.randint(low, high)))
        g, s, t = gmpy2.gcdext(e, phi_n)
        while g != 1:
            e = int(gmpy2.next_prime(random.randint(low, high)))
            g, s, t = gmpy2.gcdext(e, phi_n)
        d = s%phi_n
        
#         public_details = str(i) + '-' + str(e) + '-' + str(n)
        public_details = str(e) + '-' + str(n)
        public_details_RSA = encrypt_RSA(public_details, CA_d, CA_n)
        private_details = str(e) + '-' + str(d) + '-' + str(p) + '-' + str(q)
        private_details_RSA = encrypt_RSA(private_details, CA_d, CA_n)

        
        public_dir = open('public_dir.txt', 'a+')
        public_dir.write(str(i) + ' ' + public_details_RSA + '\n')
        public_dir.close()
        
        private_dir = open('private_' + str(i) + '.txt', 'w+')
        private_dir.write(private_details_RSA)
        private_dir.close()

In [11]:
generate_keys_CA(2)

In [12]:
cat public_dir.txt

1 gnuiergm68vr1nis-54acbqg304-zzz16h9kp3awfck4io9xej0uronrv4k4yx556iw04076ehobhmkopluzz5t5xw2b440e95j3d6-wmqoy13k28-n11ovg-xmbw2zyv0pzdhg7qfpa6i3o4qsf1vtdlnpen-opxgatnz0y55kg2gr3rlithozn70rp1ydbpslxtbx6sog3l3p--zrikwoxzw3cm04dex021qokvwukyumtpdjekbmcna04x2sty9h63ub4gxp654gzmbbulfp12zsbib5i2n7g42mg-won-3k8xps12iwfnabkwhyyid1ahpfr9bsdnsch0wd0ehd9seslnydy0961hr7kyjfumt-ev5h4ttskm77snkmusn0ptegz3-mxmc8n9uh83qk4hbz2ieikl-d4aw-pu7s2-a3sfzszz77fdvt8ltkpa6c4almqp-kuphz1vg74yo3cwsfg3aspgwy1-koyue26w52vhbs6128qrgyy1dhhnw59mca-me50wcz2ziw15s982vwj6hxlstm112-84oknfojcpjqg9rp1zpvjiimh59tb15pjnhkd3f0k06qnrpwy4jyyphlh4zt5jmc5fqyxaqqx2co5h1263hswqweblyx3rkr0plcpo4w8es-q81w-vrd87xie1ik6g7ilgvdbj-iuugz1djjq2dk9-etajef02-l1nfoh5qe8-3-atrqt5jqqmoa7c59950amfrgdosurvlxh7o9ghnau5vq2p6ye2boqxmz589ro59c-
2 evqb8759bw2ela9rsff65bi52w9j1tu2v0aje0xftzlxop3bvj41826imvmlkagoy7gqfpu-b0crrjm0o9obf0qhlzyt0eehp7c63485lomhnijmnn2mg9evdgmh39p937ow-xl33nsbe5e7zdlig--kpfhn8z9a846wrqj0e69xyx4nsi8ycarkf9py5y79jzvi7a-6q8rp8np

In [13]:
ls

'Assignment 2.pdf'   private_2.txt    Receiver.ipynb                 test.ipynb
 CA.ipynb            public_CA.txt    Sender.ipynb
 private_1.txt       public_dir.txt   send_over_unsafe_network.txt


In [14]:
cat private_2.txt

evqb8759bw2ela9rsff65bi52w9j1tu2v0aje0xftzlxop3bvj41826imvmlkagoy7gqfpu-b0crrjm0o9obf0qhlzyt0eehp7c63485lomhnijmnn2mg9evdgmh39p937ow-xl33nsbe5e7zdlig--kpfhn8z9a846wrqj0e69xyx4nsi8ycarkf9py5y79jzvi7hv5hdt93w4q8dy2i-blhgvru0bdp8m5szjg26gpkpjt4ja0rmgnzf4z-funytmx83vnhm0varn3s6ydet0uczwbceolhkgrgnoth8rztqpxqae0leyw1hcyiwe-cwt8c1rk8pmgl5vbt439oszxl5gjoxwoqavjoitw2m8lfh0gsn3ozvtgk5lc0k6e6p15y59jk2cela340lvi91p2k0va47qmbu6v0rz7u8i-kmhtundipg7gv9l5y6jdoay73lrhl6098yq-lxves8nxn97lw093hrq92pjb1mxatjjo4jeu-yrxcqop6tmuw46-48-e5q4gnqfftgm4ycwudat0xm39sz3d-b8l6fq2nq5wzubhj0ycchn1gvxuz7vxjkwky1xb6z7ieqxl1apxu0yunuc5w7v6d1ax748uev5dxf6ojsxpvhdkqwz0qf1cd5zev4cl-xm9b1944pvydd67i728atj7v6t93frz5-5j18dyp4o5s4o-nyjcwnemktwoejokntf7jmulrkisoq9hsmxm6lq62ere2t7evw82jx2euk1plp0zf-n7rij1k9w66lvwdi2w-bpcl9jy2i-bgs5xtm3p6py026l4481dbpv8bzdek4gwh5kith2wumr0rru3wrdt1ktzohnirwuhd-a4z22dqcfggv0ay54cmwk137q9bd-rjdwu2w37f4lt--dfrvsuhq2i5m4qwztjo8t2yflfy4bxe1ipyvgmt4rilaqf3rsn9vjispnb9z9jknomqgp37znhu81x7